In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("rm1000/brain-tumor-mri-scans")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\ritaj\.cache\kagglehub\datasets\rm1000\brain-tumor-mri-scans\versions\1


In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/Users/ritaj/.cache/kagglehub/datasets/rm1000/brain-tumor-mri-scans/versions/1'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/Users/ritaj/.cache/kagglehub/datasets/rm1000/brain-tumor-mri-scans/versions/1\glioma\0000.jpg
/Users/ritaj/.cache/kagglehub/datasets/rm1000/brain-tumor-mri-scans/versions/1\glioma\0001.jpg
/Users/ritaj/.cache/kagglehub/datasets/rm1000/brain-tumor-mri-scans/versions/1\glioma\0002.jpg
/Users/ritaj/.cache/kagglehub/datasets/rm1000/brain-tumor-mri-scans/versions/1\glioma\0003.jpg
/Users/ritaj/.cache/kagglehub/datasets/rm1000/brain-tumor-mri-scans/versions/1\glioma\0004.jpg
/Users/ritaj/.cache/kagglehub/datasets/rm1000/brain-tumor-mri-scans/versions/1\glioma\0005.jpg
/Users/ritaj/.cache/kagglehub/datasets/rm1000/brain-tumor-mri-scans/versions/1\glioma\0006.jpg
/Users/ritaj/.cache/kagglehub/datasets/rm1000/brain-tumor-mri-scans/versions/1\glioma\0007.jpg
/Users/ritaj/.cache/kagglehub/datasets/rm1000/brain-tumor-mri-scans/versions/1\glioma\0008.jpg
/Users/ritaj/.cache/kagglehub/datasets/rm1000/brain-tumor-mri-scans/versions/1\glioma\0009.jpg
/Users/ritaj/.cache/kagglehub/datasets/rm1000/brai

In [14]:

from tensorflow.keras.preprocessing.image import ImageDataGenerator

base_dir = '/Users/ritaj/.cache/kagglehub/datasets/rm1000/brain-tumor-mri-scans/versions/1'


In [15]:
IMG_SIZE = (128, 128)  # or 256x256 depending on your GPU memory
BATCH_SIZE = 32

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

train_generator = datagen.flow_from_directory(
    base_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    base_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)


Found 5619 images belonging to 4 classes.
Found 1404 images belonging to 4 classes.


In [17]:
from tensorflow.keras.applications import VGG16

base_model = VGG16(weights='vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5', 
                   include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))


In [19]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model

base_model = VGG16(weights='vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5', include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers on top
x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(4, activation='softmax')(x)  # 4 classes: healthy, glioma, melanoma, pituitary

model = Model(inputs=base_model.input, outputs=output)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [20]:
EPOCHS = 25
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=validation_generator
)


C:\Users\ritaj\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25
176/176 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.5860 - loss: 1.1554

C:\Users\ritaj\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


176/176 ━━━━━━━━━━━━━━━━━━━━ 225s 1s/step - accuracy: 0.5865 - loss: 1.1535 - val_accuracy: 0.8170 - val_loss: 0.4986
Epoch 2/25
176/176 ━━━━━━━━━━━━━━━━━━━━ 204s 1s/step - accuracy: 0.7588 - loss: 0.5994 - val_accuracy: 0.8568 - val_loss: 0.4129
Epoch 3/25
176/176 ━━━━━━━━━━━━━━━━━━━━ 198s 1s/step - accuracy: 0.7860 - loss: 0.5429 - val_accuracy: 0.8640 - val_loss: 0.3988
Epoch 4/25
176/176 ━━━━━━━━━━━━━━━━━━━━ 199s 1s/step - accuracy: 0.8149 - loss: 0.4713 - val_accuracy: 0.8568 - val_loss: 0.3817
Epoch 5/25
176/176 ━━━━━━━━━━━━━━━━━━━━ 212s 1s/step - accuracy: 0.8091 - loss: 0.4852 - val_accuracy: 0.8761 - val_loss: 0.3424
Epoch 6/25
176/176 ━━━━━━━━━━━━━━━━━━━━ 211s 1s/step - accuracy: 0.8105 - loss: 0.4960 - val_accuracy: 0.8768 - val_loss: 0.3173
Epoch 7/25
176/176 ━━━━━━━━━━━━━━━━━━━━ 198s 1s/step - accuracy: 0.8340 - loss: 0.4383 - val_accuracy: 0.8739 - val_loss: 0.3410
Epoch 8/25
176/176 ━━━━━━━━━━━━━━━━━━━━ 199s 1s/step - accuracy: 0.8261 - loss: 0.4530 - val_accuracy: 0.881

In [21]:
test_loss, test_acc = model.evaluate(validation_generator)
print(f"Test accuracy: {test_acc:.2f}")

# Save the model
model.save('mri_classifier_model.h5')


44/44 ━━━━━━━━━━━━━━━━━━━━ 38s 869ms/step - accuracy: 0.9030 - loss: 0.2548


Test accuracy: 0.91


In [1]:
from tensorflow.keras.models import load_model

# Load the model
model = load_model('mri_classifier_model.h5')

# Verify by printing a summary
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 128, 128, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 128, 128, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 64, 64, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 64, 64, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 32, 32, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 32, 32, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 32, 32, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 32, 32, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 16, 16, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 16, 16, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 16, 16, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 16, 16, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 8, 8, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 8, 8, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 8, 8, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 8, 8, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 4, 4, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 8192)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     2,097,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │         1,028 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 16,813,126 (64.14 MB)

 Trainable params: 2,098,436 (8.00 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

 Optimizer params: 2 (12.00 B)

In [ ]:
pip install streamlit tensorflow pillow
